In [4]:
# PROXY
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [11]:
from datasets import load_dataset
datas = load_dataset("bigbio/biored")

Repo card metadata block was not found. Setting CardData to empty.


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [12]:
from text2knowledge.strategy1 import extract_concepts

import json

In [13]:
model_name = 'mistral:latest'

In [18]:
for data in datas['test']:
    pmid = data['pmid']
    passages = data['passages']
    entities = data['entities']
    abstract = passages[1]['text'][0]
    abstract_offset = passages[1]['offsets'][0]
    metadata = {
        'pmid': pmid,
        'type': 'abstract',
    }
    entities_extracted = extract_concepts(abstract, model=model_name, metadata=metadata)

    if entities_extracted:
        entities_str = json.dumps(entities_extracted, indent=4)
        print(entities_str)
    else:
        print("No entities found.")
    break

 [
  {
      "entity": "Congenital long QT syndrome (LQTS)",
      "importance": 5,
      "category": "Disease"
  },
  {
      "entity": "In utero onset of rhythm disturbances",
      "importance": 3,
      "category": "BiologicalProcess"
  },
  {
      "entity": "Newborn patient",
      "importance": 4,
      "category": "Anatomy"
  },
  {
      "entity": "Fetal bradycardia",
      "importance": 3,
      "category": "Symptom"
  },
  {
      "entity": "2:1 atrioventricular block",
      "importance": 4,
      "category": "Pathway"
  },
  {
      "entity": "Ventricular tachycardia",
      "importance": 3,
      "category": "Symptom"
  },
  {
      "entity": "Mutational analysis and DNA sequencing",
      "importance": 4,
      "category": "MolecularFunction"
  },
  {
      "entity": "Lidocaine infusion",
      "importance": 3,
      "category": "Compound"
  },
  {
      "entity": "High dose of mexiletine",
      "importance": 3,
      "category": "Compound"
  },
  {
      "entity": "Na(

In [19]:
entities

[{'text': ['SCN5A'],
  'offsets': [[8, 13]],
  'concept_id': '6331',
  'semantic_type_id': 'GeneOrGeneProduct'},
 {'text': ['long QT syndrome'],
  'offsets': [[56, 72]],
  'concept_id': 'D008133',
  'semantic_type_id': 'DiseaseOrPhenotypicFeature'},
 {'text': ['tachycardia'],
  'offsets': [[97, 108]],
  'concept_id': 'D013610',
  'semantic_type_id': 'DiseaseOrPhenotypicFeature'},
 {'text': ['bradycardia'],
  'offsets': [[109, 120]],
  'concept_id': 'D001919',
  'semantic_type_id': 'DiseaseOrPhenotypicFeature'},
 {'text': ['long QT syndrome'],
  'offsets': [[144, 160]],
  'concept_id': 'D008133',
  'semantic_type_id': 'DiseaseOrPhenotypicFeature'},
 {'text': ['LQTS'],
  'offsets': [[162, 166]],
  'concept_id': 'D008133',
  'semantic_type_id': 'DiseaseOrPhenotypicFeature'},
 {'text': ['patient'],
  'offsets': [[292, 299]],
  'concept_id': '9606',
  'semantic_type_id': 'OrganismTaxon'},
 {'text': ['bradycardia'],
  'offsets': [[311, 322]],
  'concept_id': 'D001919',
  'semantic_type_id': 

In [20]:
data

{'pmid': '15485686',
 'passages': [{'type': 'title',
   'text': ['A novel SCN5A mutation manifests as a malignant form of long QT syndrome with perinatal onset of tachycardia/bradycardia.'],
   'offsets': [[0, 121]]},
  {'type': 'abstract',
   'text': ['OBJECTIVE: Congenital long QT syndrome (LQTS) with in utero onset of the rhythm disturbances is associated with a poor prognosis. In this study we investigated a newborn patient with fetal bradycardia, 2:1 atrioventricular block and ventricular tachycardia soon after birth. METHODS: Mutational analysis and DNA sequencing were conducted in a newborn. The 2:1 atrioventricular block improved to 1:1 conduction only after intravenous lidocaine infusion or a high dose of mexiletine, which also controlled the ventricular tachycardia. RESULTS: A novel, spontaneous LQTS-3 mutation was identified in the transmembrane segment 6 of domain IV of the Na(v)1.5 cardiac sodium channel, with a G-->A substitution at codon 1763, which changed a valine (GTG

In [21]:
abstract_offset

[122, 1815]

In [22]:
abstract

'OBJECTIVE: Congenital long QT syndrome (LQTS) with in utero onset of the rhythm disturbances is associated with a poor prognosis. In this study we investigated a newborn patient with fetal bradycardia, 2:1 atrioventricular block and ventricular tachycardia soon after birth. METHODS: Mutational analysis and DNA sequencing were conducted in a newborn. The 2:1 atrioventricular block improved to 1:1 conduction only after intravenous lidocaine infusion or a high dose of mexiletine, which also controlled the ventricular tachycardia. RESULTS: A novel, spontaneous LQTS-3 mutation was identified in the transmembrane segment 6 of domain IV of the Na(v)1.5 cardiac sodium channel, with a G-->A substitution at codon 1763, which changed a valine (GTG) to a methionine (ATG). The proband was heterozygous but the mutation was absent in the parents and the sister. Expression of this mutant channel in tsA201 mammalian cells by site-directed mutagenesis revealed a persistent tetrodotoxin-sensitive but li

In [23]:
def highlight_entities_label(s, start_idx, entities, color='#66CC66'):
    left = f"<span style='background-color: {color}'>"
    right = "</span>"
    final = ""
    last_r = 0
    for entity in entities:
        l, r = entity['offsets'][0]
        if not l >= start_idx:
            continue
        l -= start_idx
        r -= start_idx
        final += s[last_r:l]
        final += left + s[l:r] + right
        last_r = r
    final += s[r:]
    return final

In [24]:
from IPython.display import HTML

label = highlight_entities_label(abstract, abstract_offset[0], entities)
HTML(label)

In [25]:
entities_extracted

[{'entity': 'Congenital long QT syndrome (LQTS)',
  'importance': 5,
  'category': 'Disease',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'In utero onset of rhythm disturbances',
  'importance': 3,
  'category': 'BiologicalProcess',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Newborn patient',
  'importance': 4,
  'category': 'Anatomy',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Fetal bradycardia',
  'importance': 3,
  'category': 'Symptom',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': '2:1 atrioventricular block',
  'importance': 4,
  'category': 'Pathway',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Ventricular tachycardia',
  'importance': 3,
  'category': 'Symptom',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Mutational analysis and DNA sequencing',
  'importance': 4,
  'category': 'MolecularFunction',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Lidocaine infusion',
  'importance': 3,
  

In [26]:
def highlight_entities_extracted(s, entities, color='#00CCFF'):
    import re
    left = f"<span style='background-color: {color}'>"
    right = "</span>"
    final = s[:]
    for entity in entities:
        entity_text = entity['entity']
        final = re.sub('(?i)'+re.escape(entity_text), lambda m: left+entity_text+right, final)
    return final

In [27]:
predicted = highlight_entities_extracted(abstract, entities_extracted)
HTML(predicted)

In [21]:
import re

In [22]:
re.sub('hello', 'bye', 'hello HeLLo HELLO', flags=re.IGNORECASE)

'bye bye bye'

In [23]:
re.sub('Congenital long QT syndrome (LQTS)', 'hello', 'Congenital long QT syndrome (LQTS)', flags=re.IGNORECASE)

'Congenital long QT syndrome (LQTS)'

In [28]:
confusion_matrix = {"test":[0,0,0]}
categories = []

def match_entity(entity_a, entity_b):
    return entity_a['category'] == entity_b['category'] and\
model.query(entity_a['entity'], top_k=1)[2] == model.query(entity_b['entity'], top_k=1)[2]

# def macro_avg_f1_score(ans, gt):
#     # for sample_ans, sample_gt in zip(ans, gt):
#         # for category in categories:
#             # get all entities in the category
#     # ans_entities = [entity['entity']
#     #                 for entity in sample_ans if entity['category'] == category]
#     ans_entities = [entity['entity'] for entity in ans]
#     # gt_entities = [entity['text']
#     #                for entity in sample_gt if entity['category'] == category]
#     gt_entities = [entity['text'][0] for entity in gt]
#     TP = len(set(ans_entities) & set(gt_entities))
#     FP = len(set(ans_entities) - set(gt_entities))
#     FN = len(set(gt_entities) - set(ans_entities))
#     confusion_matrix["test"][0] += TP
#     confusion_matrix["test"][1] += FP
#     confusion_matrix["test"][2] += FN
#     f1_score = []
#     # for category in categories:
#     TP, FP, FN = confusion_matrix["test"]
#     precision = TP/(TP+FP) if TP+FP else float('nan')
#     recall = TP/(TP+FN) if TP+FN else float('nan')
#     f1_score.append(2*(precision * recall) / (precision + recall))
#     return sum(f1_score) / len(f1_score)

# macro_avg_f1_score(entities_extracted, entities)

In [44]:
entities_extracted

[{'entity': 'Congenital long QT syndrome (LQTS)',
  'importance': 5,
  'category': 'Disease',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'In utero onset of rhythm disturbances',
  'importance': 3,
  'category': 'BiologicalProcess',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Newborn patient',
  'importance': 4,
  'category': 'Anatomy',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Fetal bradycardia',
  'importance': 3,
  'category': 'Symptom',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': '2:1 atrioventricular block',
  'importance': 4,
  'category': 'Pathway',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Ventricular tachycardia',
  'importance': 3,
  'category': 'Symptom',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Mutational analysis and DNA sequencing',
  'importance': 4,
  'category': 'MolecularFunction',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Lidocaine infusion',
  'importance': 3,
  

In [31]:
entities[1]

{'text': ['long QT syndrome'],
 'offsets': [[56, 72]],
 'concept_id': 'D008133',
 'semantic_type_id': 'DiseaseOrPhenotypicFeature'}

In [52]:
model.query(entities_extracted[0]['entity'], top_k=10)

(array([[0.97932327, 0.9789596 , 0.9789596 , 0.97796655, 0.9778704 ,
         0.97784853, 0.97774506, 0.9776753 , 0.9774991 , 0.97747386]],
       dtype=float32),
 array([[  5550, 538983, 523573,  24920,   9590,  17247,   6439,  10193,
           9472,   9021]]),
 [5550               long QT syndrome 1
  538983    Long QT syndrome congenital
  523573    Long QT syndrome congenital
  24920     obsolete long QT syndrome 1
  9590               long qt syndrome 8
  17247              long QT syndrome 6
  6439               long QT syndrome 3
  10193              long QT syndrome 2
  9472              long QT syndrome 13
  9021              long QT syndrome 16
  Name: name, dtype: object])

In [50]:
model.query("Ventricular tachycardia", top_k=10)

(array([[1.0000001 , 1.0000001 , 1.0000001 , 1.        , 0.992821  ,
         0.992821  , 0.9922767 , 0.99227667, 0.99184203, 0.99184203]],
       dtype=float32),
 array([[488665,  10879, 535509,    623, 535610, 488926, 538123, 508571,
         535508, 466149]]),
 [488665              Ventricular tachycardia
  10879               ventricular tachycardia
  535509              Ventricular tachycardia
  623                 ventricular tachycardia
  535610               Ventricular arrhythmia
  488926               Ventricular arrhythmia
  538123    Sustained ventricular tachycardia
  508571    Sustained ventricular tachycardia
  535508             Ventricular fibrillation
  466149             Ventricular Fibrillation
  Name: name, dtype: object])

In [51]:
model.query(entities[1]['text'], top_k=10)

(array([[1.0000001 , 1.        , 1.        , 1.        , 0.9940371 ,
         0.98827636, 0.97789556, 0.97636503, 0.9754645 , 0.9740887 ]],
       dtype=float32),
 array([[534835, 448862,  11832, 486683,  21876,   2749,   9339,  30463,
          10193, 429186]]),
 [534835             Long QT syndrome
  448862             Long QT Syndrome
  11832              long QT syndrome
  486683             Long QT syndrome
  21876             short QT syndrome
  2749      familial long QT syndrome
  9339               Timothy syndrome
  30463       Long QT syndrome type 3
  10193            long QT syndrome 2
  429186             Brugada Syndrome
  Name: name, dtype: object])

In [46]:
model.query(entities[2]['text'], top_k=10)

(array([[1.0000005 , 1.0000005 , 1.0000004 , 1.0000004 , 0.98926437,
         0.9892642 , 0.9868732 , 0.9868732 , 0.98687315, 0.98687315]],
       dtype=float32),
 array([[463020,    791, 535387, 488295,    229, 534017, 486324, 444811,
         534699,    334]]),
 [463020    Tachycardia
  791       tachycardia
  535387    Tachycardia
  488295    Tachycardia
  229        arrhythmia
  534017     Arrhythmia
  486324    Hypotension
  444811    Hypotension
  534699    Hypotension
  334       hypotension
  Name: name, dtype: object])

In [10]:
set([j['semantic_type_id'] for i in datas["test"] for j in i['entities']])

{'CellLine',
 'ChemicalEntity',
 'DiseaseOrPhenotypicFeature',
 'GeneOrGeneProduct',
 'OrganismTaxon',
 'SequenceVariant'}

In [1]:
from text2knowledge.eval import BioBERT

model = BioBERT(
    model_name = 'microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext',
    index_path = '/root/text2knowledge/data/biovecs.index',
    db_path = '/root/text2knowledge/data/entities.tsv'
)

Loading model...


Some weights of the model checkpoint at microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading index...


/root/text2knowledge/text2knowledge/eval.py:20: DtypeWarning: Columns (6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  self.entities = pd.read_csv(db_path, sep='\t')


In [2]:
model.query('Congenital long QT syndrome (LQTS)')

(array([[0.97932327, 0.9789596 , 0.9789596 , 0.97796655, 0.9778704 ]],
       dtype=float32),
 array([[  5550, 523573, 538983,  24920,   9590]]),
 [5550               long QT syndrome 1
  523573    Long QT syndrome congenital
  538983    Long QT syndrome congenital
  24920     obsolete long QT syndrome 1
  9590               long qt syndrome 8
  Name: name, dtype: object])